# 3.2 - Cellule 1 : Créer une vue temporaire

In [21]:
%%sql
CREATE OR REPLACE TEMPORARY VIEW bronze_wind_power AS
SELECT *
FROM wind_power;


StatementMeta(, 0e83f34a-38b5-4506-b477-8cdee336eafe, 23, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [25]:
%%sql
USE LH_Wind_Power_Bronze.dbo;
SHOW TABLES;


StatementMeta(, 0e83f34a-38b5-4506-b477-8cdee336eafe, 28, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 2 rows and 3 fields>

In [26]:
%%sql
-- Créer une vue temporaire de la table Bronze
USE LH_Wind_Power_Bronze.dbo;

CREATE OR REPLACE TEMPORARY VIEW bronze_wind_power AS
SELECT *
FROM wind_power;


StatementMeta(, 0e83f34a-38b5-4506-b477-8cdee336eafe, 30, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

# 3.3 - Cellule 2 : Appliquer les transformations SQL

In [27]:
%%sql
-- Nettoyer et enrichir les données
CREATE OR REPLACE TEMPORARY VIEW transformed_wind_power AS
SELECT
    production_id,
    date,
    turbine_name,
    capacity,
    location_name,
    latitude,
    longitude,
    region,
    status,
    responsible_department,
    wind_direction,
    
    --  Arrondi des valeurs numériques
    ROUND(wind_speed, 2) AS wind_speed,
    ROUND(energy_produced, 2) AS energy_produced,
    
    --  Extraction des composants de date
    DAY(date) AS day,
    MONTH(date) AS month,
    QUARTER(date) AS quarter,
    YEAR(date) AS year,
    
    --  Correction du format de time
    REGEXP_REPLACE(time, '-', ':') AS time,
    
    --  Extraction des composants de temps
    CAST(SUBSTRING(time, 1, 2) AS INT) AS hour_of_day,
    CAST(SUBSTRING(time, 4, 2) AS INT) AS minute_of_hour,
    CAST(SUBSTRING(time, 7, 2) AS INT) AS second_of_minute,
    
    --  Calcul de la période de la journée
    CASE
        WHEN CAST(SUBSTRING(time, 1, 2) AS INT) BETWEEN 5 AND 11 THEN 'Morning'
        WHEN CAST(SUBSTRING(time, 1, 2) AS INT) BETWEEN 12 AND 16 THEN 'Afternoon'
        WHEN CAST(SUBSTRING(time, 1, 2) AS INT) BETWEEN 17 AND 20 THEN 'Evening'
        ELSE 'Night'
    END AS time_period

FROM bronze_wind_power;   

StatementMeta(, 0e83f34a-38b5-4506-b477-8cdee336eafe, 31, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

# 3.4 - Cellule 3 : Supprimer l'ancienne table Silver

In [28]:
%%sql
-- Supprimer l'ancienne table Silver si elle existe
USE LH_Wind_Power_Silver.dbo;
DROP TABLE IF EXISTS wind_power;


StatementMeta(, 0e83f34a-38b5-4506-b477-8cdee336eafe, 33, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

# 3.5 - Cellule 4 : Créer la nouvelle table Silver

In [29]:
%%sql
-- Créer la nouvelle table Silver avec les données transformées
USE LH_Wind_Power_Silver.dbo;

CREATE TABLE wind_power
USING delta
AS
SELECT * FROM transformed_wind_power;


StatementMeta(, 0e83f34a-38b5-4506-b477-8cdee336eafe, 35, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

# 3.6 - Cellule 5 : Vérification

In [30]:
%%sql
-- Vérifier que la table a été créée avec succès
USE LH_Wind_Power_Silver.dbo;

SELECT 
    COUNT(*) as total_rows,
    MIN(date) as min_date,
    MAX(date) as max_date,
    COUNT(DISTINCT turbine_name) as turbine_count
FROM wind_power;

StatementMeta(, 0e83f34a-38b5-4506-b477-8cdee336eafe, 37, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 1 rows and 4 fields>